In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
class LSTMCell:
    def __init__(self, input_size, hidden_size):
        self.input_size = input_size
        self.hidden_size = hidden_size

In [ ]:
        # 가중치 초기화
        self.W_f = np.random.randn(hidden_size, input_size + hidden_size) * 0.01
        self.b_f = np.zeros((hidden_size, 1))
 
        self.W_i = np.random.randn(hidden_size, input_size + hidden_size) * 0.01
        self.b_i = np.zeros((hidden_size, 1))
 
        self.W_C = np.random.randn(hidden_size, input_size + hidden_size) * 0.01
        self.b_C = np.zeros((hidden_size, 1))
 
        self.W_o = np.random.randn(hidden_size, input_size + hidden_size) * 0.01
        self.b_o = np.zeros((hidden_size, 1))

In [ ]:
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
 
    def tanh(self, x):
        return np.tanh(x)

In [ ]:
    def forward(self, x_t, h_prev, C_prev):
        """
        LSTM 셀의 순전파 연산 수행
        :param x_t: 현재 입력 (shape: input_size)
        :param h_prev: 이전 은닉 상태 (shape: hidden_size)
        :param C_prev: 이전 셀 상태 (shape: hidden_size)
        :return: h_t, C_t (현재 은닉 상태와 셀 상태)
        """
        concat = np.vstack((h_prev, x_t))  # (hidden_size + input_size, 1)
 
        # 망각 게이트
        f_t = self.sigmoid(np.dot(self.W_f, concat) + self.b_f)
        
        # 입력 게이트
        i_t = self.sigmoid(np.dot(self.W_i, concat) + self.b_i)
        C_tilde = self.tanh(np.dot(self.W_C, concat) + self.b_C)

In [ ]:
        # 새로운 셀 상태
        C_t = f_t * C_prev + i_t * C_tilde
        
        # 출력 게이트
        o_t = self.sigmoid(np.dot(self.W_o, concat) + self.b_o)
        h_t = o_t * self.tanh(C_t)
 
        return h_t, C_t, f_t, i_t, o_t, C_tilde

In [ ]:
# 입력 차원 및 은닉 상태 차원 정의
input_size = 3
hidden_size = 5
seq_length = 10  # 시퀀스 길이
 
# LSTM 셀 초기화
lstm_cell = LSTMCell(input_size, hidden_size)
 
# 초기 은닉 상태 및 셀 상태
h_t = np.zeros((hidden_size, 1))
C_t = np.zeros((hidden_size, 1))

In [ ]:
# 더미 입력 데이터 (랜덤)
np.random.seed(0)
inputs = [np.random.randn(input_size, 1) for _ in range(seq_length)]
 
# 시퀀스 처리 및 값 저장
h_states = []
C_states = []
forget_gates = []
input_gates = []
output_gates = []
cell_candidates = []

In [ ]:
for x_t in inputs:
    h_t, C_t, f_t, i_t, o_t, C_tilde = lstm_cell.forward(x_t, h_t, C_t)
    h_states.append(h_t.flatten())
    C_states.append(C_t.flatten())
    forget_gates.append(f_t.flatten())
    input_gates.append(i_t.flatten())
    output_gates.append(o_t.flatten())
    cell_candidates.append(C_tilde.flatten())

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

In [ ]:
#  애플(AAPL) 주가 데이터 다운로드
stock_data = yf.download('AAPL', start='2020-01-01', end='2024-01-01')
data = stock_data[['Close']].values
# 데이터 정규화
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data)

In [ ]:
# 시퀀스 데이터 생성
def create_sequences(data, seq_length):
   X, y = [], []
   for i inrange(len(data) - seq_length):
     X.append(data[i:i + seq_length])
     y.append(data[i + seq_length])
   return np.array(X), np.array(y)
seq_length =50
X, y = create_sequences(data_scaled, seq_length)

In [ ]:
# 훈련/테스트 데이터 분할
train_size =int(len(X) *0.8)
X_train, y_train = X[:train_size], y[:train_size]
X_test, y_test = X[train_size:], y[train_size:]

In [ ]:
# LSTM 모델 구축
model = Sequential([
   LSTM(units=50, return_sequences=True, input_shape=(seq_length, 1)),
   LSTM(units=50, return_sequences=False),
   Dense(units=25, activation='relu'),
   Dense(units=1)
])
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test), verbose=1)

In [ ]:
# 예측 수행
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)
y_test_actual = scaler.inverse_transform(y_test.reshape(-1, 1))

In [ ]:
# 결과 시각화
plt.figure(figsize=(12, 6))
plt.plot(y_test_actual, label="Actual Price", color='blue')
plt.plot(predictions, label="Predicted Price", color='red')
plt.xlabel("Time")
plt.ylabel("Stock Price")
plt.title("Apple Stock Price Prediction using LSTM")
plt.legend()
plt.show()

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding

In [ ]:
# 데이터셋 (Alice in Wonderland 텍스트 일부)
text = """Alice was beginning to get very tired of sitting by her sister on the bank, 
and of having nothing to do: 
once or twice she had peeped into the book her sister was reading, 
but it had no pictures or conversations in it, 
"and what is the use of a book," 
thought Alice "without pictures or conversation?" """

In [ ]:
# 토큰화
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1
 
# 시퀀스 데이터 생성
input_sequences = []
words = text.split()
for i in range(1, len(words)):
    n_gram_sequence = words[:i+1]
    encoded = tokenizer.texts_to_sequences([" ".join(n_gram_sequence)])[0]
    input_sequences.append(encoded)

In [ ]:
# 패딩 적용
max_length = max([len(seq) for seq in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_length, padding='pre')
 
# X, y 분할
X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

In [ ]:
# LSTM 모델 구축
model = Sequential([
    Embedding(total_words, 10, input_length=max_length-1),
    LSTM(100),
    Dense(total_words, activation='softmax')
])
 
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=500, verbose=1)

In [ ]:
# 텍스트 생성 함수
def generate_text(seed_text, next_words=10):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_length-1, padding='pre')
        predicted = np.argmax(model.predict(token_list), axis=-1)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

In [ ]:
# 텍스트 생성 함수
def generate_text(seed_text, next_words=10):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_length-1, padding='pre')
        predicted = np.argmax(model.predict(token_list), axis=-1)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text
# 예제 실행
print(generate_text("Alice was beginning", next_words=10))

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [ ]:
# IMDB 데이터셋 로드
num_words = 10000  # 상위 10,000개의 단어만 사용
max_length = 200  # 리뷰 길이 제한
 
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=num_words)
 
# 패딩 적용 (모든 리뷰를 동일한 길이로 맞춤)
X_train = pad_sequences(X_train, maxlen=max_length, padding='post')
X_test = pad_sequences(X_test, maxlen=max_length, padding='post')

In [ ]:
# LSTM 기반 감성 분석 모델 구축
model = Sequential([
    Embedding(input_dim=num_words, output_dim=32, input_length=max_length),
    LSTM(64),
    Dense(1, activation='sigmoid')  # 긍정(1) / 부정(0) 분류
])
 
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=5, batch_size=128, validation_data=(X_test, y_test))

In [ ]:
# 정확도 평가
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.2f}")